## Assignment 3 - Generative Adversarial Network Programming (Full points: 100)

This assignment is more like a tutorial. 

### About Grading
(10 points)
Essentially, you need to go through all the coding details we provide to you, and experiment with it. We will grade you by checking the history to see if you have run the code. 
(60 points) You need to complete certain code segments. If you don't completely go through the tutorial, you might fail this part.
(30 points)
Finally, you should adjust the hyperparameters to achieve a reasonable result (a lower printed loss) in order to receive a high grade.

### Content
In this assignment (with tutorial), you're going to create your first generative adversarial network (GAN). Specifically, you need to build and train a GAN that can generate hand-written images of digits (0 to 9). You will be using PyTorch in this specialization, so if you're not familiar with this framework, you may find the [PyTorch documentation](https://pytorch.org/docs/stable/index.html) useful. The hints will also often include links to relevant documentation. 

### Warning
If you encounter code errors (bugs), even in the tutorial, please attempt to resolve them independently, search for solutions using search engines such as Google, or seek advice from fellow classmates (plagiarism is not permitted). Failing to rectify bugs within the assignment or merely writing code without producing executable results will result in a significant deduction of points.

### Objectives
1.   Build the generator and discriminator components of a GAN from scratch.
2.   Create generator and discriminator loss functions.
3.   Train your GAN and visualize the generated images.


### Key Concepts & Review

##### MNIST Dataset
The training images your discriminator will be using is from a dataset called MNIST. It contains 60,000 images of handwritten digits, from 0 to 9. (Google it if you want to see the details.)

You may notice that the images are quite pixelated -- this is because they are all only 28 x 28! The small size of its images makes MNIST ideal for simple training. Additionally, these images are also in black-and-white so only one dimension, or "color channel").

##### Tensor
You will represent the data using [tensors](https://pytorch.org/docs/stable/tensors.html). Tensors are a generalization of matrices: for example, a stack of three matrices with the amounts of red, green, and blue at different locations in a 64 x 64 pixel image is a tensor with the shape 3 x 64 x 64.

Tensors are easy to manipulate and supported by [PyTorch](https://pytorch.org/), the library you will be using. Feel free to explore them more, but you can imagine these as multi-dimensional matrices or vectors!

##### Batches
While you could train your model after generating one image, it is extremely inefficient and leads to less stable training. In GANs, and in deep learning in general, you will process multiple images per training step. These are called batches.

This means that your generator will generate an entire batch of images and receive the discriminator's feedback on each before updating the model. The same goes for the discriminator, it will calculate its loss on the entire batch of generated images as well as on the reals before the model is updated.

In [1]:
import torch

print(torch.__version__)

if torch.cuda.is_available():
    print("GPU is available.")
else:
    print("GPU is not available.")


2.0.1
GPU is not available.


# Tutorial (10 points)

You will begin by importing some useful packages and the dataset you will use to build and train your GAN.

In [2]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) # Set for testing purposes, please do not change!

def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''
    image_unflat = image_tensor.view(-1, *size).detach()#.cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generator
The first step is to build the generator component.

You will start by creating a function to make a single layer/block for the generator's neural network. Each block should include a [linear transformation](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) to map to another shape, a [batch normalization](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) for stabilization, and finally a non-linear activation function (you use a [ReLU here](https://pytorch.org/docs/master/generated/torch.nn.ReLU.html)) so the output can be transformed in complex ways.

In [3]:
def get_generator_block(input_dim, output_dim):
    '''
    Function for returning a block of the generator's neural network
    given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a generator neural network layer, with a linear transformation 
          followed by a batch normalization and then a relu activation
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )

In [4]:
# Verify the generator block function
def test_gen_block(in_features, out_features, num_test=1000):
    block = get_generator_block(in_features, out_features)

    # Check the three parts
    assert len(block) == 3
    assert type(block[0]) == nn.Linear
    assert type(block[1]) == nn.BatchNorm1d
    assert type(block[2]) == nn.ReLU
    
    # Check the output shape
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)
    assert tuple(test_output.shape) == (num_test, out_features)
    assert test_output.std() > 0.55
    assert test_output.std() < 0.65

test_gen_block(25, 12)
test_gen_block(15, 28)
print("Success!")

Success!


Now you can build the generator class. It will take 3 values:

*   The noise vector dimension
*   The image dimension
*   The initial hidden dimension

Using these values, the generator will build a neural network with 5 layers/blocks. Beginning with the noise vector, the generator will apply non-linear transformations via the block function until the tensor is mapped to the size of the image to be outputted (the same size as the real images from MNIST). You will need to fill in the code for final layer since it is different than the others. The final layer does not need a normalization or activation function, but does need to be scaled with a [sigmoid function](https://pytorch.org/docs/master/generated/torch.nn.Sigmoid.html). 

Finally, you are given a forward pass function that takes in a noise vector and generates an image of the output dimension using your neural network.

<details>

<summary>
<font size="3" color="green">
<b>Optional hints for <code><font size="4">Generator</font></code></b>
</font>
</summary>

1. The output size of the final linear transformation should be im_dim, but remember you need to scale the outputs between 0 and 1 using the sigmoid function.
2. [nn.Linear](https://pytorch.org/docs/master/generated/torch.nn.Linear.html) and [nn.Sigmoid](https://pytorch.org/docs/master/generated/torch.nn.Sigmoid.html) will be useful here. 
</details>


In [5]:
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
          (MNIST images are 28 x 28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, z_dim=10, im_dim=784, hidden_dim=128):
        super(Generator, self).__init__()
        # Build the neural network
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        return self.gen(noise)
    
    def get_gen(self):
        '''
        Returns:
            the sequential model
        '''
        return self.gen

In [6]:
# Verify the generator class
def test_generator(z_dim, im_dim, hidden_dim, num_test=10000):
    gen = Generator(z_dim, im_dim, hidden_dim).get_gen()
    
    # Check there are six modules in the sequential part
    assert len(gen) == 6
    test_input = torch.randn(num_test, z_dim)
    test_output = gen(test_input)

    # Check that the output shape is correct
    assert tuple(test_output.shape) == (num_test, im_dim)
    assert test_output.max() < 1, "Make sure to use a sigmoid"
    assert test_output.min() > 0, "Make sure to use a sigmoid"
    assert test_output.std() > 0.05, "Don't use batchnorm here"
    assert test_output.std() < 0.15, "Don't use batchnorm here"

test_generator(5, 10, 20)
test_generator(20, 8, 24)
print("Success!")

Success!


## Noise
To be able to use your generator, you will need to be able to create noise vectors. The noise vector z has the important role of making sure the images generated from the same class don't all look the same -- think of it as a random seed. You will generate it randomly using PyTorch by sampling random numbers from the normal distribution. Since multiple images will be processed per pass, you will generate all the noise vectors at once.

Note that whenever you create a new tensor using torch.ones, torch.zeros, or torch.randn, you either need to create it on the target device, e.g. `torch.ones(3, 3, device=device)`, or move it onto the target device using `torch.ones(3, 3).to(device)`. You do not need to do this if you're creating a tensor by manipulating another tensor or by using a variation that defaults the device to the input, such as `torch.ones_like`. In general, use `torch.ones_like` and `torch.zeros_like` instead of `torch.ones` or `torch.zeros` where possible.

<details>

<summary>
<font size="3" color="green">
<b>Optional hint for <code><font size="4">get_noise</font></code></b>
</font>
</summary>

1. 
You will probably find [torch.randn](https://pytorch.org/docs/master/generated/torch.randn.html) useful here.
</details>

In [7]:
def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim),
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''
    # NOTE: To use this on GPU with device='cuda', make sure to pass the device 
    # argument to the function you use to generate the noise.
    return torch.randn(n_samples, z_dim, device=device)

In [8]:
# Verify the noise vector function
def test_get_noise(n_samples, z_dim, device='cpu'):
    noise = get_noise(n_samples, z_dim, device)
    
    # Make sure a normal distribution was used
    assert tuple(noise.shape) == (n_samples, z_dim)
    assert torch.abs(noise.std() - torch.tensor(1.0)) < 0.01
    assert str(noise.device).startswith(device)

test_get_noise(1000, 100, 'cpu')
if torch.cuda.is_available():
    test_get_noise(1000, 32, 'cuda')
    print("Success on gpu!")
else:
    print("Success!")

Success!


## Discriminator
The second component that you need to construct is the discriminator. As with the generator component, you will start by creating a function that builds a neural network block for the discriminator.

*Note: You use leaky ReLUs to prevent the "dying ReLU" problem, which refers to the phenomenon where the parameters stop changing due to consistently negative values passed to a ReLU, which result in a zero gradient. You will learn more about this in the following lectures!* 

In [9]:
def get_discriminator_block(input_dim, output_dim):
    '''
    Discriminator Block
    Function for returning a neural network of the discriminator given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a discriminator neural network layer, with a linear transformation 
          followed by an nn.LeakyReLU activation with negative slope of 0.2 
          (https://pytorch.org/docs/master/generated/torch.nn.LeakyReLU.html)
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True)
    )

In [10]:
# Verify the discriminator block function
def test_disc_block(in_features, out_features, num_test=10000):
    block = get_discriminator_block(in_features, out_features)

    # Check there are two parts
    assert len(block) == 2
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)

    # Check that the shape is right
    assert tuple(test_output.shape) == (num_test, out_features)
    
    # Check that the LeakyReLU slope is about 0.2
    assert -test_output.min() / test_output.max() > 0.1
    assert -test_output.min() / test_output.max() < 0.3
    assert test_output.std() > 0.3
    assert test_output.std() < 0.5

test_disc_block(25, 12)
test_disc_block(15, 28)
print("Success!")

Success!


Now you can use these blocks to make a discriminator! The discriminator class holds 2 values:

*   The image dimension
*   The hidden dimension

The discriminator will build a neural network with 4 layers. It will start with the image tensor and transform it until it returns a single number (1-dimension tensor) output. This output classifies whether an image is fake or real. Note that you do not need a sigmoid after the output layer since it is included in the loss function. Finally, to use your discrimator's neural network you are given a forward pass function that takes in an image tensor to be classified.


In [11]:
class Discriminator(nn.Module):
    '''
    Discriminator Class
    Values:
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
            (MNIST images are 28x28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, im_dim=784, hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            # Hint: You want to transform the final output into a single value,
            #       so add one more linear map.
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):
        '''
        Function for completing a forward pass of the discriminator: Given an image tensor, 
        returns a 1-dimension tensor representing fake/real.
        Parameters:
            image: a flattened image tensor with dimension (im_dim)
        '''
        return self.disc(image)
    
    # Needed for grading
    def get_disc(self):
        '''
        Returns:
            the sequential model
        '''
        return self.disc

In [12]:
# Verify the discriminator class
def test_discriminator(z_dim, hidden_dim, num_test=100):
    
    disc = Discriminator(z_dim, hidden_dim).get_disc()

    # Check there are three parts
    assert len(disc) == 4

    # Check the linear layer is correct
    test_input = torch.randn(num_test, z_dim)
    test_output = disc(test_input)
    assert tuple(test_output.shape) == (num_test, 1)
    
    # Make sure there's no sigmoid
    assert test_input.max() > 1
    assert test_input.min() < -1

test_discriminator(5, 10)
test_discriminator(20, 8)
print("Success!")

Success!


# (60 points) Complete the missing code
# (30 points) Turning hyperparameters to get a great performance (with lower printed loss).

Now you can put it all together!
First, you will set your parameters:
  *   criterion: the loss function
  *   n_epochs: the number of times you iterate through the entire dataset when training
  *   z_dim: the dimension of the noise vector
  *   display_step: how often to display/visualize the images
  *   batch_size: the number of images per forward/backward pass
  *   lr: the learning rate
  *   device: the device type, here using a GPU (which runs CUDA), not CPU

Next, you will load the MNIST dataset as tensors using a dataloader.

In [13]:
# Set your parameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 100
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.00001

In [14]:
# Load MNIST dataset as tensors
dataloader = DataLoader(
    MNIST('.', download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)

device = 'cpu' # or 'cuda'

Now, you can initialize your generator, discriminator, and optimizers. Note that each optimizer only takes the parameters of one particular model, since we want each optimizer to optimize only one of the models.

In [15]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

Before you train your GAN, you will need to create functions to calculate the discriminator's loss and the generator's loss. This is how the discriminator and generator will know how they are doing and improve themselves. Since the generator is needed when calculating the discriminator's loss, you will need to call .detach() on the generator result to ensure that only the discriminator is updated!

Remember that you have already defined a loss function earlier (`criterion`) and you are encouraged to use `torch.ones_like` and `torch.zeros_like` instead of `torch.ones` or `torch.zeros` to set the labels. If you use `torch.ones` or `torch.zeros`, you'll need to pass `device=device` to them.

In [16]:
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    '''
    Return the loss of the discriminator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        real: a batch of real images
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        disc_loss: a torch scalar loss value for the current batch
    '''
    #     These are the steps you will need to complete:
    #       1) Create noise vectors and generate a batch (num_images) of fake images. 
    #            Make sure to pass the device argument to the noise.
    #       2) Get the discriminator's prediction of the fake image 
    #            and calculate the loss. Don't forget to detach the generator!
    #            (Remember the loss function you set earlier -- criterion. You need a 
    #            'ground truth' tensor in order to calculate the loss. 
    #            For example, a ground truth tensor for a fake image is all zeros.)
    #       3) Get the discriminator's prediction of the real image and calculate the loss.
    #       4) Calculate the discriminator's loss by averaging the real and fake loss
    #            and set it to disc_loss.
    #     Note: Please do not use concatenation in your solution. The tests are being updated to 
    #           support this, but for now, average the two losses as described in step (4).
    #     *Important*: You should NOT write your own loss function here - use criterion(pred, true)!
    #### START CODE HERE ####
    noise = get_noise(num_images, z_dim, device)
    fake_image_gen = gen(noise).detach()  # Detach the generator to prevent gradients from flowing back.
    fake_image_pred = disc(fake_image_gen)
    fake_image_loss = criterion(fake_image_pred, torch.zeros_like(fake_image_pred))
    real_image_pred = disc(real)
    real_image_loss = criterion(real_image_pred, torch.ones_like(real_image_pred))
    disc_loss = (fake_image_loss + real_image_loss) / 2
    #### END CODE HERE ####
    return disc_loss

In [17]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    '''
    Return the loss of the generator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        gen_loss: a torch scalar loss value for the current batch
    '''
    #     These are the steps you will need to complete:
    #       1) Create noise vectors and generate a batch of fake images. 
    #           Remember to pass the device argument to the get_noise function.
    #       2) Get the discriminator's prediction of the fake image.
    #       3) Calculate the generator's loss. Remember the generator wants
    #          the discriminator to think that its fake images are real
    #     *Important*: You should NOT write your own loss function here - use criterion(pred, true)!

    #### START CODE HERE ####
    noise = get_noise(num_images, z_dim, device)
    fake_image_gen = gen(noise)
    fake_image_pred = disc(fake_image_gen)
    gen_loss = criterion(fake_image_pred, torch.ones_like(fake_image_pred))
    #### END CODE HERE ####
    return gen_loss

Finally, you can put everything together! For each epoch, you will process the entire dataset in batches. For every batch, you will need to update the discriminator and generator using their loss. Note that you may see a loss to be greater than 1, this is okay since binary cross entropy loss can be any positive number for a sufficiently confident wrong guess. 

It’s also often the case that the discriminator will outperform the generator, especially at the start, because its job is easier. It's important that neither one gets too good (that is, near-perfect accuracy), which would cause the entire model to stop learning. Balancing the two models is actually remarkably hard to do in a standard GAN.

In addition, be warned that this runs very slowly on a CPU. One way to run this more quickly is to use Google Colab: 

1.   Download the .ipynb
2.   Upload it to Google Drive and open it with Google Colab
3.   Make the runtime type GPU (under “Runtime” -> “Change runtime type” -> Select “GPU” from the dropdown)
4.   Replace `device = "cpu"` with `device = "cuda"`
5.   Make sure your `get_noise` function uses the right device -->

In [18]:
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
gen_loss = False
error = False

for epoch in range(n_epochs):
    # Dataloader returns the batches
    for real, _ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradients before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)

        # Update gradients
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        ### Update generator ###
        #     Hint: This code will look a lot like the discriminator updates!
        #     These are the steps you will need to complete:
        #       1) Zero out the gradients.
        #       2) Calculate the generator loss, assigning it to gen_loss.
        #       3) Backprop through the generator: update the gradients and optimizer.
        #### START CODE HERE ####
        # Zero out the gradients.
        gen_opt.zero_grad()

        # Calculate the generator loss
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)

        # Backprop through the generator and update the gradients and optimizer.
        gen_loss.backward()
        gen_opt.step()
        #### END CODE HERE ####

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")

            # Generate fake images for visualization
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            # You can display the generated images here using 'show_tensor_images(fake)' and 'show_tensor_images(real)'.

            # Reset the mean loss values
            mean_generator_loss = 0
            mean_discriminator_loss = 0

        cur_step += 1

  9%|▉         | 42/469 [00:00<00:07, 59.28it/s]

Epoch 1, step 500: Generator loss: 1.3942078813314447, discriminator loss: 0.41791092991828965


 16%|█▌        | 74/469 [00:01<00:06, 60.14it/s]

Epoch 2, step 1000: Generator loss: 1.7532655124664298, discriminator loss: 0.2792234902083876


 22%|██▏       | 102/469 [00:01<00:06, 59.83it/s]

...

 10%|█         | 49/469 [00:00<00:07, 58.41it/s]

Epoch 98, step 46000: Generator loss: 1.9646357102394092, discriminator loss: 0.31785580441355715


 17%|█▋        | 79/469 [00:01<00:06, 59.14it/s]

Epoch 99, step 46500: Generator loss: 1.9618738436698915, discriminator loss: 0.3097116082012652


100%|██████████| 469/469 [00:08<00:00, 58.48it/s]


Tuning Hyperparameters

In [19]:
# Set your parameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 100
z_dim = 64
display_step = 500
batch_size = 64
lr = 0.0002

In [20]:
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
gen_loss = False
error = False

for epoch in range(n_epochs):
    # Dataloader returns the batches
    for real, _ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradients before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)

        # Update gradients
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        ### Update generator ###
        #     Hint: This code will look a lot like the discriminator updates!
        #     These are the steps you will need to complete:
        #       1) Zero out the gradients.
        #       2) Calculate the generator loss, assigning it to gen_loss.
        #       3) Backprop through the generator: update the gradients and optimizer.
        #### START CODE HERE ####
        # Zero out the gradients.
        gen_opt.zero_grad()

        # Calculate the generator loss
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)

        # Backprop through the generator and update the gradients and optimizer.
        gen_loss.backward()
        gen_opt.step()
        #### END CODE HERE ####

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")

            # Generate fake images for visualization
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            # You can display the generated images here using 'show_tensor_images(fake)' and 'show_tensor_images(real)'.

            # Reset the mean loss values
            mean_generator_loss = 0
            mean_discriminator_loss = 0

        cur_step += 1

  8%|▊         | 38/469 [00:00<00:07, 59.17it/s]

Epoch 1, step 500: Generator loss: 1.9790336453914643, discriminator loss: 0.3127773903012278


 16%|█▌        | 74/469 [00:01<00:06, 59.10it/s]

Epoch 2, step 1000: Generator loss: 1.9874166448116308, discriminator loss: 0.30615846979618067


 21%|██▏       | 100/469 [00:01<00:06, 60.20it/s]

...

 10%|█         | 49/469 [00:00<00:07, 59.04it/s]

Epoch 98, step 46000: Generator loss: 1.189287846565246, discriminator loss: 0.49443673473596594


 16%|█▋        | 77/469 [00:01<00:06, 56.19it/s]

Epoch 99, step 46500: Generator loss: 1.2540588521957403, discriminator loss: 0.47353887957334523


100%|██████████| 469/469 [00:08<00:00, 57.77it/s]


In [21]:
import itertools
import sys
import torch
import torch.nn as nn
from tqdm import tqdm

hyperparameter_values = {
    'z_dim': [32, 64, 128],
    'hidden_dim': [64, 128, 256],
    'lr': [0.0002, 0.0001],
}

best_hyperparameters = None
best_generator_loss = float('inf')

original_stdout = sys.stdout
with open('output.txt', 'w') as f:
    sys.stdout = f

    for z_dim, hidden_dim, lr in itertools.product(hyperparameter_values['z_dim'],
                                                   hyperparameter_values['hidden_dim'],
                                                   hyperparameter_values['lr']):
        print(f"Training with hyperparameters: z_dim={z_dim}, hidden_dim={hidden_dim}, lr={lr}")

        # Set your parameters
        criterion = nn.BCEWithLogitsLoss()
        n_epochs = 20
        display_step = 500
        batch_size = 128

        cur_step = 0
        mean_generator_loss = 0
        mean_discriminator_loss = 0
        gen_loss = False
        error = False

        # Initialize your model, optimizer, and dataloader with the current hyperparameters
        gen = Generator(z_dim).to(device)  # Assuming 'device' is defined
        gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
        disc = Discriminator().to(device)  # Assuming 'device' is defined
        disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

        for epoch in range(n_epochs):
            # Dataloader returns the batches
            for real, _ in tqdm(dataloader):
                cur_batch_size = len(real)

                # Flatten the batch of real images from the dataset
                real = real.view(cur_batch_size, -1).to(device)

                ### Update discriminator ###
                # Zero out the gradients before backpropagation
                disc_opt.zero_grad()

                # Calculate discriminator loss
                disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)

                # Update gradients
                disc_loss.backward(retain_graph=True)

                # Update optimizer
                disc_opt.step()

                ### Update generator ###
                # Zero out the gradients.
                gen_opt.zero_grad()

                # Calculate the generator loss
                gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)

                # Backprop through the generator and update the gradients and optimizer.
                gen_loss.backward()
                gen_opt.step()

                # Keep track of the average discriminator loss
                mean_discriminator_loss += disc_loss.item() / display_step

                # Keep track of the average generator loss
                mean_generator_loss += gen_loss.item() / display_step

                ### Visualization code ###
                if cur_step % display_step == 0 and cur_step > 0:
                    print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")

                    # Generate fake images for visualization
                    fake_noise = get_noise(cur_batch_size, z_dim, device=device)
                    fake = gen(fake_noise)
                    # You can display the generated images here using 'show_tensor_images(fake)' and 'show_tensor_images(real)'.

                    # Reset the mean loss values
                    mean_generator_loss = 0
                    mean_discriminator_loss = 0

                cur_step += 1

        if mean_generator_loss < best_generator_loss:
            best_generator_loss = mean_generator_loss
            best_hyperparameters = {'z_dim': z_dim, 'hidden_dim': hidden_dim, 'lr': lr}

    print("Best Hyperparameters:", best_hyperparameters)

sys.stdout = original_stdout

100%|██████████| 469/469 [00:08<00:00, 57.38it/s]


Unfortunately, due to time, I was not able to finish training my Hyperparameter tuning iteration as it took longer to train than I had expected; however, if I were able to complete training it, I would be able to see the Generator Loss and Discriminator Loss for 100 Epochs of the 18 different combinations of Hyperparameters as well as which combination did the best. Unfortunately, my computer does not let me use my GPU, and I had issues with Google Colab. As a result of this time limit, I changed the number of Epochs from 100 to 20. I have included the code for the 100 Epochs below just to include it.

In [22]:
# import itertools

# hyperparameter_values = {
#     'z_dim': [32, 64, 128],
#     'hidden_dim': [64, 128, 256],
#     'lr': [0.0002, 0.0001],
# }

# best_hyperparameters = None
# best_generator_loss = float('inf')

# for z_dim, hidden_dim, lr in itertools.product(hyperparameter_values['z_dim'],
#                                                hyperparameter_values['hidden_dim'],
#                                                hyperparameter_values['lr']):
#     print(f"Training with hyperparameters: z_dim={z_dim}, hidden_dim={hidden_dim}, lr={lr}")

#     criterion = nn.BCEWithLogitsLoss()
#     n_epochs = 100
#     display_step = 500
#     batch_size = 128

#     cur_step = 0
#     mean_generator_loss = 0
#     mean_discriminator_loss = 0
#     gen_loss = False
#     error = False

#     # Initialize your model, optimizer, and dataloader with the current hyperparameters
#     gen = Generator(z_dim).to(device)
#     gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
#     disc = Discriminator().to(device) 
#     disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

#     for epoch in range(n_epochs):
#         # Dataloader returns the batches
#         for real, _ in tqdm(dataloader):
#             cur_batch_size = len(real)

#             # Flatten the batch of real images from the dataset
#             real = real.view(cur_batch_size, -1).to(device)

#             ### Update discriminator ###
#             # Zero out the gradients before backpropagation
#             disc_opt.zero_grad()

#             # Calculate discriminator loss
#             disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)

#             # Update gradients
#             disc_loss.backward(retain_graph=True)

#             # Update optimizer
#             disc_opt.step()

#             ### Update generator ###
#             # Zero out the gradients.
#             gen_opt.zero_grad()

#             # Calculate the generator loss
#             gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)

#             # Backprop through the generator and update the gradients and optimizer.
#             gen_loss.backward()
#             gen_opt.step()

#             # Keep track of the average discriminator loss
#             mean_discriminator_loss += disc_loss.item() / display_step

#             # Keep track of the average generator loss
#             mean_generator_loss += gen_loss.item() / display_step

#             ### Visualization code ###
#             if cur_step % display_step == 0 and cur_step > 0:
#                 print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")

#                 # Generate fake images for visualization
#                 fake_noise = get_noise(cur_batch_size, z_dim, device=device)
#                 fake = gen(fake_noise)
#                 # You can display the generated images here using 'show_tensor_images(fake)' and 'show_tensor_images(real)'.

#                 # Reset the mean loss values
#                 mean_generator_loss = 0
#                 mean_discriminator_loss = 0

#             cur_step += 1

#     if mean_generator_loss < best_generator_loss:
#         best_generator_loss = mean_generator_loss
#         best_hyperparameters = {'z_dim': z_dim, 'hidden_dim': hidden_dim, 'lr': lr}

# print("Best Hyperparameters:", best_hyperparameters)